# DeepOF notebook for Busquets-Garcia Lab
Welcome to this notebook, where you'll find all the code necessary to analyze your DeepLabCut (DLC) output with DeepOF. Don't worry! You don't need notions of coding with Python, as you'll need to run the cells of this notebook. Just keep reading, and I'll guide you through the following steps.

## Import the necessary packages and directories
First, run the following cell to import the necessary packages for your analysis.

In [ ]:
import numpy as np
import pandas as pd
import pickle
import os
import deepof.data

Now it's time to add your directories. Modify the following to fit the directories where you store your files:
NOTE: run the corrected videos with the DLC files of the corrected videos.

In [ ]:
directory_output = '//FOLDER/becell/Lab Projects/ERCstG_HighMemory/Data/Marc/1) SOC/2023-10 - Alzheimer/Females/DeepOF/'
directory_dlc = '//FOLDER/becell/Lab Projects/ERCstG_HighMemory/Data/Marc/1) SOC/2023-10 - Alzheimer/Females/DeepOF/dlc/'
directory_videos = '//FOLDER/becell/Lab Projects/ERCstG_HighMemory/Data/Marc/1) SOC/2023-10 - Alzheimer/Females/DeepOF/corrected/'

## Create your DeepOF project folder
Now that you have all set, it's time to start your analysis. Just run the following cell to create a folder with your DeepOF project (where you'll perform your study). But before, take into consideration:
- To draw an area, click on the corners of the arena. Press **d** to delete and **q** once you have finished.
- The first edge that you draw will be used to escalate from pixels to millimeters. The default value is 200 mm, corresponding to the longest edge of the polybox.
- We will perform the arena detection semi-automatically. You'll have to draw the arena once, and the rest will be predicted based on your first attempt. The program will create a folder named *Arena_detection* to store samples of the automatic detection so you can check if everything went smoothly.
- If you want to perform a manual drawing in each video (it may be tedious!), you can do it by switching **manual=False** to **manual=True** in the function below.
- If you have a problem drawing the arena of a specific video, don't worry! Note the number of the video causing problems, draw any shape, and go on. After the analysis is completed, run the next cell indicating the video's name and re-draw the arena's shape.

In [ ]:
from master_script import creating_deepof_project
my_deepof_project = creating_deepof_project(directory_output, directory_dlc, directory_videos, manual=False, scale=200)

In [ ]:
# To re-draw the shape of an arena
my_deepof_project.edit_arenas(videos=['20230728_Marc_ERC SOC S1_Males_box ef_05_01_1',
                                      '20230728_Marc_ERC SOC S2_Males_box ab_04_01_1',
                                      '20230728_Marc_ERC SOC S2_Males_box ab_06_01_1',
                                      '20230728_Marc_ERC SOC S2_Males_box cd_04_01_1'
                                      ])

Finally, you may add some conditions to your experiment (e.g., a column named *protocol* in which you indicate the group to which the video belongs). To do so, first, you must create a CSV file (you can do that in Excel and then export it to CSV) and store it in your _directory_output_. You cave an example named ``conditions.csv`` in this repository.

In [ ]:
my_deepof_project.load_exp_conditions(directory_output + "conditions.csv")

In [ ]:
# Optional: in case you want to check that your conditions have been correctly imported:
from master_script import check_conditions
check_conditions(directory_output, my_deepof_project, col_name='protocol')

## Perform a supervised analysis
With the supervised analysis, you can predict different pre-defined behaviors. We start supposing that you created a DeepOF project (that is to say, that you ran the cells from the section above). In case you have a DeepOF project and you want to load it, run the following cell:

In [ ]:
my_deepof_project = deepof.data.load_project(directory_output + "deepof_tutorial_project")

Once loaded with a DeepOF project, it's time to run the supervised analysis! For the sake of time, we'll store the output from the supervised analysis in a pickle file named **supervised_annotation.pkl**. Then, you can load it in the future and start working with it immediately without re-doing the analysis. In summary, to perform a new supervised analysis, run the first cell; to load an existing one, the second cell.

In [ ]:
# Perform a new supervised analysis
supervised_annotation = my_deepof_project.supervised_annotation()
with open(directory_output + 'supervised_annotation.pkl', 'wb') as file:
    pickle.dump(supervised_annotation, file)

In [ ]:
# Alternatively, open an existing supervised analysis
with open(directory_output + 'supervised_annotation.pkl', 'rb') as file:
    supervised_annotation = pickle.load(file)

Considering that we have other data sources, we might think it would be interesting to merge all of them. Running the following cell, we can combine the output of Júlia's and Rémi's script for immobility analysis into the data of DeepOF. You must enter a folder directory where you store the pickle files obtained after the immobility analysis.

In [ ]:
from master_script import update_supervised_annotation_with_immobility

directory_path = '//FOLDER/becell/Lab Projects/ERCstG_HighMemory/Data/Marc/1) SOC/2023-09 - Young males/DeepOF/Data/pickles/'
update_supervised_annotation_with_immobility(supervised_annotation, directory_path)

## Perform an unsupervised analysis
With an unsupervised analysis, you can identify patterns in your data without labeling or a pre-existing hypothesis. Therefore, you can use it to explore your data and discover new relationships you wouldn't have noticed! First, we'll preprocess our data; DeepOF will calculate the centered and aligned coordinates, speeds, and distances corresponding between the animal parts.

In [ ]:
from master_script import graph_dataset_function
graph_preprocessed_coords, adj_matrix, to_preprocess, global_scaler = graph_dataset_function(my_deepof_project)

Now, we will embed our data with deep clustering methods. The core idea of deep clustering is to embed our preprocessed data with a neural network and retrieve a set of embeddings per time point, each assigned to a cluster. If you have already trained a model, set **pre_trained=True**** instead of *False*.

In [ ]:
from master_script import train_model_function
trained_model = train_model_function(my_deepof_project, graph_preprocessed_coords, adj_matrix, pre_trained=False)